In [125]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [126]:
N_EPOCHS = 50
N_TRIALS = 3
BATCH_SIZE = 256

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [127]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

train_size = x_train.shape[0]

In [128]:
noise = 30
x_train = np.clip(x_train + np.random.normal(0, noise, x_train.shape), 0, 255) / 255
x_val = np.clip(x_val + np.random.normal(0, noise, x_val.shape), 0, 255) / 255
x_test = np.clip(x_test + np.random.normal(0, noise, x_test.shape), 0, 255) / 255

In [129]:
results = {}

# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]

In [130]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [131]:
for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=10, **models_hyperparameters[test_model])
        
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)
            samples_ids = np.random.choice(range(train_size), n_samples, replace=False)

            model.fit(
                x_train[samples_ids], y_train[samples_ids], 
                validation_data=(x_val, y_val),
                epochs=1, batch_size=BATCH_SIZE,
                verbose=1, callbacks=[early_stopping]
            )
        
        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
        
        model_scores.append(accuracy)

    results[test_model] = model_scores

test_model_1


  0%|          | 0/3 [00:00<?, ?it/s]

6/6 [==============================] - 0s 13ms/step - loss: 1.3288 - accuracy: 0.5027


 33%|███▎      | 1/3 [00:50<01:41, 50.61s/it]

6/6 [==============================] - 0s 10ms/step - loss: 1.4661 - accuracy: 0.4673


 67%|██████▋   | 2/3 [02:01<01:02, 62.81s/it]

6/6 [==============================] - 0s 15ms/step - loss: 1.4742 - accuracy: 0.4607


100%|██████████| 3/3 [03:08<00:00, 62.89s/it]


In [133]:
results

{'test_model_1': [0.5026666522026062, 0.4673333466053009, 0.4606666564941406]}